<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| Fabrice Jimenez | <a href="https://supaerodatascience.github.io/deep-learning/">https://supaerodatascience.github.io/deep-learning/</a>

# Dimensionality Reduction - Practical Application

This notebook is following the progression of the Dimensionality Reduction class. It provides practical illustrations in Python  to understand the notions we have seen in this course.

<div class="alert alert-warning">
Author: Fabrice JIMENEZ
    
Link to course materials: https://github.com/jfabrice/ml-class-dimensionality-reduction
</div>

## Preliminary loading with Google Colab

If you are using this notebook with Google Colab, please execute first the following cells, to retrieve the GitHub repository content. Otherwise, ignore these cells and move to the next section.

In [ ]:
import os
os.mkdir('data')

In [ ]:
!wget -c https://github.com/SupaeroDataScience/deep-learning/blob/main/DR/data/mnist_test.csv?raw=true -O data/mnist_test.csv


## Imports and Loading Dataset

The dataset we will use for this application is the famous MNIST dataset (handwritten digits). We use the CSV version of this dataset available here: https://pjreddie.com/projects/mnist-in-csv/ 

We will keep only the mnist_test.csv file, containing 10 000 gray-scale images of dimension 28 x 28.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('data/mnist_test.csv', header=None)
df['pixels'] = df.index.map(lambda x: np.array(df.iloc[x][1:]))
dropcols = df.columns[(df.columns != 0) * (df.columns != 'pixels')]
df.drop(dropcols, axis=1, inplace=True)
df.columns = ['label','pixels']
print(df.shape)

In [ ]:
df.head()

In [ ]:
fig = plt.imshow(df['pixels'][2].reshape(28,28), cmap='gray')

## Dimensionality reduction

In this section we will think about how to distinguish the characteristics of different handwritten digits, starting only from the raw values of the pixels as features.

Let's consider 3 digits. Keep them as they are for the moment, you will have time at the end to play with these parameters.

In [ ]:
## Digits considered here ##
labels = [1,6,8]
colors = ['red', 'blue', 'green']

X = np.array([df['pixels'][i] for i in df.index if df['label'][i] in labels])
y = np.array([df['label'][i] for i in df.index if df['label'][i] in labels])

print('X shape: '+str(X.shape))
print('y shape: '+str(y.shape))

We have 28 x 28 = 784 features. It is a high dimension (~ same order of magnitude as the number of points). 

<div class="alert alert-warning">
Question: How would you visualize the behavior of the different classes (digits), or find a direction, important features contributing to these classes?
</div>

In this case, we cannot rely on univariate analysis: it is quite clear that the value of a given pixel on the image will not determine by itself the number which is written. You need to study the relationship between the pixel values: let's see how the methods to reduce dimensionality we saw during this course can help us distinguish noise, correlation and information patterns!

#### Principal Components Analysis: linear approach

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
## Computing the PCA
pca = PCA(n_components=X.shape[1])
XPCA = pca.fit_transform(X)

In [ ]:
## Choose the axes you want to visualize
component_x = 1
component_y = 2

## Plotting
fig, ax = plt.subplots()
for i in range(len(labels)):
    ax.scatter(XPCA[y == labels[i],component_x-1], XPCA[y == labels[i],component_y-1], c=colors[i], label=labels[i], alpha=0.4)

l = ax.legend()

<div class="alert alert-warning">
Question: Try to visualize different axes and find the ones who allow (or not) to distinguish the classes.
</div>

#### t-distributed Stochastic Neighbor Embedding: non-linear approach

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
## Computing the t-SNE 2D projection

## Parameters with real influence on the accuracy
perplexity = 30
learning_rate = 200
n_iter = 1000

tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter)
XTSNE = tsne.fit_transform(X)

In [ ]:
## Plotting
fig, ax = plt.subplots()
for i in range(len(labels)):
    ax.scatter(XTSNE[y == labels[i],0], XTSNE[y == labels[i],1], c=colors[i], label=labels[i], alpha=0.4)
l = ax.legend()

<div class="alert alert-warning">
Question: Try to visualize the projection by choosing different parameter values. What is the advantage of t-SNE over PCA? What is the inconvenient?
</div>

## Impact on supervised model performance

In this section, we consider a supervised Machine Learning model to predict the class of an image (which digit it corresponds to). We will study the impact of dimensionality reduction on 3 classifiers which have different mechanisms:
- Naive Bayes Classifier (nb)
- SVM Classifier (svm)
- Random Forest Classifier (rf)

We can give them as features 3 different inputs:
- All the pixel values (raw)
- The n first PCA components (10 for example)
- The 2 dimensions from t-SNE projection (tsne)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
def fit_my_model(model, features, test_size):
    
    ## Building train and test sets from features
    if features == 'raw':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    elif features == 'tsne':
        X_train, X_test, y_train, y_test = train_test_split(XTSNE, y, test_size=test_size)
    else:
        X_train, X_test, y_train, y_test = train_test_split(XPCA[:,:features], y, test_size=test_size)

    print("Training samples: "+str(X_train.shape[0]))
    print("Testing samples: "+str(X_test.shape[0]))
    print("Number of features: "+str(X_train.shape[1]))
    
    ## Fitting model
    if model == 'nb':
        clf = GaussianNB()
    elif model == 'svm':
        clf = SVC(gamma='auto')
    elif model == 'rf':
        clf = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=None, max_features=np.min([10, X_train.shape[1]]))

    clf.fit(X_train, y_train)
    
    ## Printing scores
    learningScore = clf.score(X_train, y_train)
    generalizationScore = clf.score(X_test, y_test)
    print('Learning score: '+str(learningScore))
    print('Generalization score: '+str(generalizationScore))
    
    return generalizationScore

<div class="alert alert-warning">
Question: Experiment in the cell below by using the function fit_my_model, to check the scores for different configurations
</div>

In [ ]:
model = 'svm' ## svm, nb, rf
features = 'raw' ## raw, tsne, or Integer corresponding to the n first PCA components
test_size = 0.2

score = fit_my_model(model, features, test_size)

<div class="alert alert-warning">
Question: What is the effect of dimensionality reduction on the 3 classifiers? Try to explain why with your intuition...
</div>

Let's test all the configurations and build a result table with test scores, to help you answer this question.

In [ ]:
comparison_table = pd.DataFrame(columns = ['raw', 'pca', 'tsne'], index=['svm', 'nb', 'rf'])
for f in comparison_table.columns:
    for m in comparison_table.index:
        print(f + ' - ' + m)
        comparison_table.loc[m,f] = fit_my_model(m, f if f!='pca' else 10, 0.2)

In [ ]:
## Heatmap with summarized results
fig = sns.heatmap(comparison_table.astype('float'), annot=True, cmap='Reds')

<div class="alert alert-warning">
Question: What conclusions can you give? If you have time left, you can try with different sets of digits.
</div>

These examples concern supervised classifiers only. Can you imagine the impact on other tasks: regression, clustering, anomaly detection... ?

## Autoencoders

In this section we will build step by step an Autoencoder architecture, train it and evaluate it on MNIST data. First, it will be a standard fully connected autoencoder, then a variational autoencoder in the next section.

To prepare this work, let's import the appropriate libraries and use the full MNIST dataset from Keras, as Neural Networks will need more data for training.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = np.array([x.flatten() for x in X_train]).astype("float32") / 255.
X_test = np.array([x.flatten() for x in X_test]).astype("float32") / 255.
print(X_train.shape)
print(X_test.shape)

In this case we have normalized the pixel values between 0 and 1, to help neural network modelling (sigmoid) and training.

### Standard Autoencoder

In this first part, let's implement a standard autoencoder (fully-connected) with 1 hidden layer (200 units, ReLU activation) before the latent space (10 units, ReLU activation). Let's begin with the separate architectures for encoder and decoder. Be careful with the final layer activation!

In [ ]:
###### CODE HERE ######

# Encoder architecture
encoder_inputs = keras.Input(shape=(784,))
hidden1 = ###
latent_space = ###

encoder = keras.Model(encoder_inputs, latent_space, name="encoder")
#######################

In [ ]:
###### CODE HERE ######

# Decoder architecture
decoder_inputs = keras.Input(shape=(10,))
hidden2 = ###
decoder_outputs = ###

decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")
#######################

Now, we combine them to build the full autoencoder.

In [ ]:
# Combining architectures
outputs = decoder(latent_space)
autoencoder = keras.Model(encoder_inputs, outputs, name="autoencoder")

In [ ]:
autoencoder.summary()

Now we need to define the loss function for training. In the case of MNIST images, we can use a pixelwise binary cross-entropy, summed up over the image. We could also use a mean squared error, let's leave it as an exercise.

In [ ]:
# Loss function definition
reconstruction_loss = keras.losses.binary_crossentropy(encoder_inputs, outputs) * 784

autoencoder_loss = K.mean(reconstruction_loss)

In [ ]:
# Compiling the model
autoencoder.add_loss(autoencoder_loss)
autoencoder.compile(optimizer="adam")

In [ ]:
###### CODE HERE ######

# Fitting the model
history = autoencoder.fit(###, ###,
          epochs=30,
          batch_size=128,
          shuffle=True,
          validation_data=(###, ###))

#######################

In [ ]:
# Visualizing the training and validation losses
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

We seem to obtain a pretty reasonable training process! Let's visualize the quality of reconstructed images.

In [ ]:
n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # Original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Reconstructed images
    ax = plt.subplot(2, n, i + 1 + n)
    encoded_img = encoder.predict(np.array([X_test[i]]).reshape(1,784))
    decoded_img = decoder.predict(encoded_img).reshape(28, 28)
    plt.imshow(decoded_img, cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

We can see that some details are lost, but the global shapes are correctly reconstructed.

### Variational Autoencoder

In this second part, let's implement a variational autoencoder. Let's begin by defining the specific layer we will use to sample values from the gaussian distribution defined by the latent space means and standard deviations (z_mean, z_logvar).

In [ ]:
###### CODE HERE ######

# Coding the specific sampling layer as a Keras Layer object
class Sampling(layers.Layer):

    def call(self, inputs):
        z_mean, z_logvar = inputs

        nbatch = K.shape(z_mean)[0]
        ndim = K.shape(z_mean)[1]

        std = K.exp(z_logvar)
        eps = K.random_normal(shape=(nbatch,ndim), mean=0., stddev=0.1)
        
        z = ###

        return z

#######################

Let's write now the separate encoder and decoder architectures. Be careful, no activation function for mean and logvar computation...

In [ ]:
###### CODE HERE ######

# Encoder architecture
encoder_inputs = keras.Input(shape=(784,))
hidden1 = layers.Dense(200, activation="relu")(encoder_inputs)

z_mean = ###
z_logvar = ###

# Sampling
z = Sampling()([z_mean, z_logvar])

encoder = keras.Model(encoder_inputs, z, name="encoder")

#######################

In [ ]:
# Decoder architecture
decoder_inputs = keras.Input(shape=(10,))
decoder_hidden = layers.Dense(200, activation="relu")(decoder_inputs)
decoder_outputs = layers.Dense(784, activation="sigmoid")(decoder_hidden)

decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")

Now, we combine them to build the full autoencoder.

In [ ]:
# Combining architectures
outputs = decoder(z)
vae = keras.Model(encoder_inputs, outputs, name="vae")

In [ ]:
vae.summary()

Now we need to define the loss function for training. As reconstruction loss, we can still use the pixelwise binary cross-entropy, summed up over the image. In the case of VAE, there is an additional term to the loss: the Kullback-Leibler divergence. Write this new term, using the keras backend functions: K.square, K.exp, K.sum...

In [ ]:
###### CODE HERE ######

# Loss function definition
reconstruction_loss = keras.losses.binary_crossentropy(encoder_inputs, outputs) * 784

kl_loss = ###

vae_loss = K.mean(reconstruction_loss + kl_loss)

#######################

In [ ]:
# Compiling the model
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [ ]:
# Fitting the model
history = vae.fit(X_train, X_train,
          epochs=30,
          batch_size=128,
          shuffle=True,
          validation_data=(X_test, X_test))

In [ ]:
# Visualizing the training and validation losses
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

We seem to obtain a pretty reasonable training process! Let's visualize the quality of reconstructed images.

In [ ]:
n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # Original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Reconstructed images
    ax = plt.subplot(2, n, i + 1 + n)
    encoded_img = encoder.predict(np.array([X_test[i]]).reshape(1,784))
    decoded_img = decoder.predict(encoded_img).reshape(28, 28)
    plt.imshow(decoded_img, cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

We can see that some details are lost, but the global shapes are correctly reconstructed.

Since a VAE uses a generative model, we can use the decoder to build fake images, and we can see that the latent space is continuous! Let's sample some random images in a given range of latent space values.

In [ ]:
n = 15  # figure with 15x15 generated images
figure = np.zeros((28 * n, 28 * n))

# We sample n images within [-15, 15] standard deviations, around 0 mean
grid_x = np.linspace(-15, 15, n)
grid_y = np.linspace(-15, 15, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        
        # We sample the latent space over the 2 first neurons, feel free to change that to other pairs!
        z_sample = np.array([[xi, yi, 0, 0, 0, 0, 0, 0, 0, 0]])
        x_decoded = decoder.predict(z_sample)

        digit = x_decoded[0].reshape(28, 28)
        figure[i * 28: (i + 1) * 28,
               j * 28: (j + 1) * 28] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap="gray")
plt.show()

<div class="alert alert-warning">
Exercise: if you had to detect an abnormal image (for example: not a digit) in this 1D dataset, how would you do?
</div>

Try to introduce a gray-scale 28x28 image of your choice in the dataset (picture of your cat...). Shuffle the dataset, and test your anomaly detection method to find it!

## Conclusion

Always think first about the principles, the intuition, the qualitative aspect behind all the available functions you can find online.

- Many people can chain very complex algorithms together and get results which might be relevant for a problem.
- Only a few can make the right choices to quickly optimize the resolution of a problem and assess its feasability...